In [2]:
import pandas as pd

df_train = pd.read_pickle('../data/train.pkl')
df_test = pd.read_pickle('../data/test.pkl')

In [13]:
df_train.index = range(len(df_train))
print(df_train)
print(df_test)

            userid      docid      timestamp  network  refresh  position  \
0       2409825960  463552562  1624869556203        5       19       207   
1       1305114184  463843741  1624838347464        5        4        58   
2       1384753520  462965256  1624656823023        5        4         9   
3       1793347126  463964144  1624865763852        5        1        20   
4       2429494146  463966614  1624930276109        5        1        25   
...            ...        ...            ...      ...      ...       ...   
816706  1377807890  463472453  1624750686655        5       10        42   
816707  1781149274  463439945  1624778543969        2        2        25   
816708  1801510258  462742027  1624671676147        5        5        12   
816709  2444319116  463173395  1624855979077        2        4        35   
816710  2412452152  463456346  1624864247820        2       64       106   

        click  duration                               dt        date  
0           0   

In [27]:
print(df_train.loc[2])
print()
print(df_test.loc[2])

userid                             1384753520
docid                               462965256
timestamp                       1624656823023
network                                     5
refresh                                     4
position                                    9
click                                       0
duration                                    0
dt           2021-06-26 05:33:43.023000+08:00
date                               2021-06-26
Name: 2, dtype: object

id                                          2
userid                             1637010070
docid                               464687570
timestamp                       1625053546711
network                                     2
refresh                                    10
dt           2021-06-30 19:45:46.711000+08:00
date                               2021-06-30
Name: 2, dtype: object


In [130]:
doc_info = pd.read_pickle('../data/doc_info.pkl')
user_info = pd.read_pickle('../data/user_info.pkl')

In [19]:
print(user_info.loc[1])
print()
print(doc_info.loc[1])

userid      394666
device         160
os               0
province       207
city           571
age              0
gender           2
Name: 1, dtype: int64

docid                  334493096
title          汽车点评网友“毛蒙”分享的小视频“
pubtime            1563404900000
picnum                         0
category1st                   23
category2nd                  115
keyword           https:8.135272
Name: 1, dtype: object


In [31]:
for index,i in enumerate(user_info['userid']):
#     print(i)
    if i in ['1305114184','1623790870','1384753520','1637010070']:
        print(user_info.loc[index])
    

In [30]:
for index,i in enumerate(doc_info['docid']):
#     print(i)
    if i in ['462965256','464687570']:
        print(doc_info.loc[index])

docid                                                  462965256
title                                       小沈龙发文咒骂辛巴!低俗网红真的该管管了
pubtime                                            1624550400000
picnum                                                         4
category1st                                                   11
category2nd                                                   57
keyword        价值观:8.343280,低俗化:10.175401,剧本:10.201841,吐槽:7.9...
Name: 92809, dtype: object
docid                                                  464687570
title                          林青霞住11亿豪宅,装修就花了6个亿,亲姐却在河南拿450元退休金
pubtime                                            1625018600000
picnum                                                         6
category1st                                                   11
category2nd                                                   60
keyword        亲姐妹:7.262579,优衣库:8.142245,养父:11.374247,剧本:7.23...
Name: 323223, dtype: object


In [32]:
# 准备数据
target='click'    # Disbursed的值就是二元分类的输出
# IDcol = 'ID'


In [60]:
from sklearn.model_selection import train_test_split
x_columns = [x for x in df_train.columns if x not in [target,'dt','date','duration','position']]
X = df_train[x_columns]      # 样本
y = df_train[target]    # 标签

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=9)


In [61]:
from sklearn.ensemble import GradientBoostingClassifier
gbm0 = GradientBoostingClassifier(random_state=9)
gbm0.fit(x_train, y_train)

GradientBoostingClassifier(random_state=9)

In [ ]:

# auc_score=roc_auc_score(y_true,y_pred)

In [62]:
from sklearn.metrics import roc_auc_score,accuracy_score
print("tr-accuracy: %.4g" % accuracy_score(y_train.values, gbm0.predict(x_train)))      # Accuracy : 0.9855
print("tr-AUC: %f" % roc_auc_score(y_train, gbm0.predict_proba(x_train)[:, 1]))      # AUC Score (Train): 0.910597

print("val-accuracy: %.4g" % accuracy_score(y_val, gbm0.predict(x_val)))      # Accuracy : 0.9825
print("val-AUC: %f" % roc_auc_score(y_val, gbm0.predict_proba(x_val)[:, 1]))      # AUC Score (Train): 0.827217

tr-accuracy: 0.8547
tr-AUC: 0.653316
val-accuracy: 0.854
val-AUC: 0.649249


In [63]:
print(x_train[0:1])

            userid      docid      timestamp  network  refresh
220166  2441338242  463448887  1624852490544        5        0


In [64]:
gbm0.predict_proba(x_train[0:1])

array([[0.75579758, 0.24420242]])

In [65]:
x_columns_0 = [x for x in df_test.columns if x not in [target,'dt','date']]
X_0 = df_test[x_columns_0]      # 样本
print(X_0[0:1])

   id      userid      docid      timestamp  network  refresh
0   0  2204868548  464495940  1625011774995        2        4


In [67]:
del X_0['id']  # 删除A列，会就地修改
print(X_0[0:1])

       userid      docid      timestamp  network  refresh
0  2204868548  464495940  1625011774995        2        4


In [74]:
gbm0.predict_proba(X_0[0:1])

array([[0.86696635, 0.13303365]])

In [75]:
pre_0=gbm0.predict_proba(X_0)[:, 1]

In [76]:
print(pre_0[0])

0.13303364524547762


In [77]:
def store_list(lsit,text_path):
    ff = open(text_path, encoding='utf-8', mode='w')
    for line_list in lsit:
        ff.write(str(line_list))  # 写入一个新文件中
        ff.write("\n")
store_list(pre_0,'../data/pre_0.txt')

In [82]:
df_train_user=pd.merge(df_train,user_info,on='userid')
print(df_train_user.loc[0])


userid                             2409825960
docid                               463552562
timestamp                       1624869556203
network                                     5
refresh                                    19
position                                  207
click                                       0
duration                                    0
dt           2021-06-28 16:39:16.203000+08:00
date                               2021-06-28
device                                   1620
os                                          0
province                                  195
city                                      370
age                                         0
gender                                      2
Name: 0, dtype: object


In [132]:

all_keywords = set()

def get_all_keyword(x):
    global max_len, min_len
    if x == '':
        return

    splts = x.split(',')

    for sp in splts:
        keyword = sp.split(':')[0]
        all_keywords.add(keyword)


doc_info['keyword'].fillna('', inplace=True)
doc_info['keyword'].apply(get_all_keyword)

keyword2id = dict(zip(all_keywords, range(0, len(all_keywords))))

def keyword_map(x):
    if x == '':
        return '0'*len(all_keywords)

    keys = []
    for sp in x.split(','):
        keyword = sp.split(':')[0]
        key_id=keyword2id[keyword]
        keys.append(key_id)
    ret=''
    for i in range(len(all_keywords)):
        if i in keys:
            ret+='1'
            continue
        else:
            ret+='0'
            
    return ret


doc_info['keyword'] = doc_info['keyword'].apply(keyword_map)

KeyboardInterrupt: 

In [ ]:
print(len(doc_info['keyword'][1]))

In [ ]:
print(doc_info.loc[0])

In [125]:
keyword_l=[]
for i in doc_info['keyword']:
    keyword_l.append(i)


In [126]:
print(keyword_l[1])

[72302]


In [123]:
import numpy as np

keyword_l=np.array(keyword_l)
print(keyword_l[1])

[72302]
[72302]


In [89]:
del doc_info['title']
print(doc_info.loc[0])

docid              325279629
pubtime        1624889500000
picnum                     0
category1st                0
category2nd                0
keyword                   []
Name: 0, dtype: object


In [92]:
doc_info['docid'] = doc_info['docid'].astype('int')
df_train_user_doc=pd.merge(df_train_user,doc_info,on='docid')
print(df_train_user_doc.loc[0])
print(df_train_user_doc)

userid                                                2409825960
docid                                                  463552562
timestamp                                          1624869556203
network                                                        5
refresh                                                       19
position                                                     207
click                                                          0
duration                                                       0
dt                              2021-06-28 16:39:16.203000+08:00
date                                                  2021-06-28
device                                                      1620
os                                                             0
province                                                     195
city                                                         370
age                                                            0
gender                   

In [93]:
del df_train_user_doc['dt']
del df_train_user_doc['date']
print(df_train_user_doc.loc[0])


userid                                                2409825960
docid                                                  463552562
timestamp                                          1624869556203
network                                                        5
refresh                                                       19
position                                                     207
click                                                          0
duration                                                       0
device                                                      1620
os                                                             0
province                                                     195
city                                                         370
age                                                            0
gender                                                         2
pubtime                                            1624700600000
picnum                   

In [96]:
# df_test
df_test_user=pd.merge(df_test,user_info,on='userid')
df_test_user_doc=pd.merge(df_test_user,doc_info,on='docid')

In [106]:
# del df_test_user_doc['id']
df_test_user_doc.index = range(len(df_test_user_doc))

In [107]:
print(df_train_user_doc)
print(df_test_user_doc)


            userid      docid      timestamp  network  refresh  position  \
0       2409825960  463552562  1624869556203        5       19       207   
1       1333578642  463552562  1624789087645        2        7        39   
2       2414005114  463552562  1624810077750        5       24        11   
3       1083181620  463552562  1624744948477        2       12       124   
4       2039950314  463552562  1624904353183        2       16       172   
...            ...        ...            ...      ...      ...       ...   
816706  2396415728  462966593  1624578325117        2        8        24   
816707  2264501418  463554415  1624725221146        5       15       144   
816708    83662116  463188052  1624701491646        5        5        64   
816709  1519085878  463668159  1624789668176        2        1        20   
816710  1152601138  464105307  1624958394000        2       11       121   

        click  duration  device  os  province  city  age  gender  \
0           0      

In [98]:
df_test_user_doc.sort_values("id",inplace=True)
print(df_test_user_doc)

          id      userid      docid      timestamp  network  refresh  \
0          0  2204868548  464495940  1625011774995        2        4   
133        1  1623790870  464553715  1625066054894        2        2   
234        2  1637010070  464687570  1625053546711        2       10   
237        3  1634251474  464661865  1625047014335        2       10   
370        4  2352721164  464692079  1625035383911        2       11   
...      ...         ...        ...            ...      ...      ...   
233    49995  1640585246  464553715  1625036568698        2        1   
4474   49996  1987843480  464382231  1625009469794        2        1   
5891   49997  2391080328  464617167  1625027566512        5        1   
42467  49998   125699342  464172118  1625034692217        2       11   
3676   49999  2154437014  464225813  1625008637470        2        3   

                                    dt        date  device  os  province  \
0     2021-06-30 08:09:34.995000+08:00  2021-06-30    1479 

In [111]:
from sklearn.model_selection import train_test_split
x_columns = [x for x in df_train_user_doc.columns if x not in ['click','dt','date','duration','position']]
X = df_train_user_doc[x_columns]      # 样本
y = df_train_user_doc['click']    # 标签

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=9)


In [112]:
from sklearn.ensemble import GradientBoostingClassifier
gbm0 = GradientBoostingClassifier(random_state=9)
gbm0.fit(x_train, y_train)

ValueError: setting an array element with a sequence.

In [110]:
from sklearn.metrics import roc_auc_score,accuracy_score
print("tr-accuracy: %.4g" % accuracy_score(y_train.values, gbm0.predict(x_train)))      # Accuracy : 0.9855
print("tr-AUC: %f" % roc_auc_score(y_train, gbm0.predict_proba(x_train)[:, 1]))      # AUC Score (Train): 0.910597

print("val-accuracy: %.4g" % accuracy_score(y_val, gbm0.predict(x_val)))      # Accuracy : 0.9825
print("val-AUC: %f" % roc_auc_score(y_val, gbm0.predict_proba(x_val)[:, 1]))      # AUC Score (Train): 0.827217

tr-accuracy: 0.8547
tr-AUC: 0.653316
val-accuracy: 0.854
val-AUC: 0.649249


In [ ]:
x_columns_0 = [x for x in df_test.columns if x not in ['click','dt','date','duration','position']]
X_0 = df_test[x_columns_0]      # 样本
print(X_0[0:1])

In [ ]:
y_pre_0=gbm0.predict_proba(X_0)[:, 1]